###  Юнит 3. Введение в машинное обучение 

#### Preproject (предвариетльный анализ)

Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

 - **Restaurant_id** — идентификационный номер ресторана / сети ресторанов;
 - **City** — город, в котором находится ресторан;
 - **Cuisine Style** — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
 - **Ranking** — место, которое занимает данный ресторан среди всех ресторанов своего города;
 - **Rating** — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
 - **Price Range** — диапазон цен в ресторане;
 - **Number of Reviews** — количество отзывов о ресторане;
 - **Reviews** — данные о двух отзывах, которые отображаются на сайте ресторана;
 - **URL_TA** — URL страницы ресторана на TripAdvosor;
 - **ID_TA** — идентификатор ресторана в базе данных TripAdvisor.

Задачу, которая стоит перед вами, можно свести к трём пунктам:

 - Удалить из датафрейма столбцы, данные в которых представлены не числами (это вы уже сделали, и нужно просто повторить знакомые действия, но в этот раз выполнить данный шаг в последнюю очередь).
 - Избавиться от пропущенных (None) значений (на предыдущем шаге мы делали это самым грубым из всех возможных способов; сейчас попробуем подойти к процессу более гибко).
 - Создать новые столбцы с данными, используя для этого информацию, содержащуюся в других столбцах датафрейма (например, можно добавить столбец, сообщающий, сколько дней прошло со дня публикации последнего отзыва, отображённого на сайте).
 
С другой стороны, в этом задании масса подводных камней.

In [20]:
# импорт библиотек
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split # Загружаем специальный инструмент для разбивки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 
from collections import Counter
import os
# import re
# import math
# import copy

In [22]:
df = pd.read_csv('main_task.csv')
display(df.head())

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [23]:
df.columns = df.columns.str.replace(' ','_')
df

,Restaurant_id,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


### ТРЕНИРОВКА

### 2.1 Задание 1
Какие столбцы НЕ содержат пропущенных (None) значений?

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine_Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price_Range        26114 non-null  object 
 6   Number_of_Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [25]:
df.columns[df.isna().sum() == 0]

Index(['Restaurant_id', 'City', 'Ranking', 'Rating', 'Reviews', 'URL_TA',
       'ID_TA'],
      dtype='object')

### 2.2 Задание 2
В каких столбцах данные хранятся в числовом формате?  
Ответ:  
- Ranking 
- Rating   
- Number of Reviews

In [26]:
df.columns[df.dtypes != np.object]

Index(['Ranking', 'Rating', 'Number_of_Reviews'], dtype='object')

### 2.3 Задание 3
В каких столбцах хранящиеся данные представляют собой список?

In [17]:
for vol in df.iloc[0]:
    if type(vol) == list: print(vol.index)


Ответ:  
   - Ни в одном

In [37]:
# сделаем из кода построения модели функцию, так как нам придется ее повторять 
def first_model(df):
    # Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
    X = df.drop(['Rating'], axis = 1)  
    y = df['Rating']  
    
    # Наборы данных с меткой "train" будут использоваться для обучения модели, 
    # "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

    # Создаём модель  
    regr = RandomForestRegressor(n_estimators=100)  
    
    # Обучаем модель на тестовом наборе данных  
    regr.fit(X_train, y_train)  
    
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
    # Предсказанные значения записываем в переменную y_pred  
    y_pred = regr.predict(X_test) 

    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), 
    # и смотрим насколько они в среднем отличаются  
    # Метрика называется Mean Absolute Error (MAE) и показывает 
    # среднее отклонение предсказанных значений от фактических.  
    print('MAE:', metrics.mean_absolute_error(y_test, y_pred)) 

In [27]:
# запускаем функцию с построением модели и оценкой МАЕ
first_model(df)

NameError: name 'first_model' is not defined

### 3.1 Вопрос для размышления  
По какой причине во время обучения модели возникла ошибка Value Error?  

***Ответ:*** - Ошибка возникла при обработке столбца со строковыми значениями. Следовательно, для обучения модели все данные должны быть переданы только в числовом формате. 


In [29]:
# заполняем пропуски 
# сначала заполним числовой ряд средними значениями 
df['Number_of_Reviews'] = df['Number_of_Reviews'].fillna(df['Number_of_Reviews'].mean())

In [30]:
# проверяем заполнение
df['Number_of_Reviews'].isnull().sum()

0

In [31]:
# теперь запольним Price Range
# сначала заменим все значения на числовые 
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price_Range']=df['Price_Range'].map(lambda x: dic_value_Price.get(x,x))
# потом заменим на самое популярное
df['Price_Range'] = df['Price_Range'].fillna(df["Price_Range"].value_counts().idxmax())

In [32]:
# проверяем заполнение
df['Price_Range'].isnull().sum()

0

In [33]:
# City переводим в числа с помощью метода get_dummies
df_city_dummies = pd.get_dummies(df['City'])
df = pd.concat([df,df_city_dummies], axis=1)

In [34]:
# на этом тренировочном этапе удалим все столбцы, 
# которые сложно заполнить или перевести в числа
df.drop(['Restaurant_id', 'City', 'Cuisine_Style', 'Reviews', 'URL_TA', 'ID_TA'], axis=1, inplace=True)

In [35]:
df

,Ranking,Rating,Price_Range,Number_of_Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570.0,3.5,2.0,194.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,2.0,10.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,353.0,4.5,3.0,688.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3458.0,5.0,2.0,3.0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,621.0,4.0,2.0,84.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,4.5,2.0,79.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39996,6341.0,3.5,2.0,542.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
39997,1652.0,4.5,2.0,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
39998,641.0,4.0,2.0,70.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
# повторяем построение модели
first_model(df)

MAE: 0.21757349999999998


### 4.1 Вопросы о ценах
1. Сколько вариантов непустых значений встречается в столбце Price Range?  
Ответ: 3

2. Как в датафрейме обозначается самый низкий уровень цен?  
Ответ: $

3. Как в датафрейме обозначается самый высокий уровень цен?  
Ответ: `$$$$`

4. Сколько ресторанов относятся к среднему ценовому сегменту?

In [40]:
# перед решением следующих задач скачиваем заново датасет, 
# потому что мы уже изменили первоначальный датасет
df = pd.read_csv('main_task.csv')

temp = len(df[df['Price Range'] == '$$ - $$$'])
print(round(temp,0))

18412


### 4.2 Вопрос о городах
Сколько городов представлено в наборе данных?

In [43]:
temp = df['City'].nunique()
temp

31

### 4.3 Вопросы о кухнях
1. Сколько типов кухонь представлено в наборе данных?  
2. Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов. 
3. Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [45]:
# заполним пропуски в Cuisine Style
# на этапе обучения просто поставим какую-нибудь "другую" кухню
df['Cuisine Style'] = df['Cuisine Style'].fillna("['Other']")
df['Cuisine Style'] = df['Cuisine Style'].str.findall(r"'(\b.*?\b)'") 

# создадим список списков кухонь
temp_list = df['Cuisine Style'].tolist()

# но у нас получился список списков, с которым работать неудобно
# напишем функцию распаковки списка списков 
# взяла данный подход из задания 1, потому что он достаточно быстрый)
def list_unrar(list_of_lists):
    result=[]
    for lst in list_of_lists:
      result.extend(lst)
    return result

#  записываем распакованный список в Counter
temp_counter=Counter(list_unrar(temp_list))

In [50]:
result1 = len(temp_counter) - 1  # минусуем нашу одну "другую" кухню
result1

125

In [48]:
result2 = temp_counter.most_common()[0][0]
result2

'Vegetarian Friendly'

In [51]:
# сделаем аналог get_dummies
for cuisine in temp_counter:
    df[cuisine] = df['Cuisine Style'].apply(lambda x: 1 if cuisine in x else 0 ).astype('float64')

In [52]:
# чтобы не удалять столбец и не создавать новый 
# запишем  количество кухонь в текущий
df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: len(x)).astype('float64')

In [54]:
result3 = df['Cuisine Style'].mean()
print(round(result3, 1))

2.6


### 4.4 Вопросы об отзывах
1. Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.
2. Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [55]:
# заполним пустые значения в Reviews
df['Reviews'] = df['Reviews'].fillna("['no_Reviews']")
# найдем даты и запишем в новый столбец в формате даты
df['date_of_Review'] = df['Reviews'].str.findall('\d+/\d+/\d+')

In [57]:
# проверим
with pd.option_context('display.max_columns', None):
    display(df.head())

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,European,French,International,Other,Japanese,Sushi,Asian,Grill,Vegetarian Friendly,Vegan Options,Gluten Free Options,German,Central European,Italian,Pizza,Fast Food,Mediterranean,Spanish,Healthy,Cafe,Thai,Vietnamese,Bar,Pub,Chinese,British,Polish,Fusion,Dutch,Mexican,Venezuelan,South American,Soups,Belgian,Steakhouse,Latin,Barbecue,Argentinean,Irish,Seafood,Swiss,Portuguese,Contemporary,Wine Bar,Greek,Central American,Indian,Middle Eastern,Turkish,Hungarian,Pakistani,Peruvian,Delicatessen,Eastern European,Swedish,Scandinavian,Tibetan,Nepali,Korean,Southwestern,Czech,American,Slovenian,Balti,Street Food,Diner,Brew Pub,Caribbean,Austrian,Moroccan,Halal,Lebanese,Russian,African,Ethiopian,Egyptian,Danish,Brazilian,Ecuadorean,Israeli,Kosher,Gastropub,Australian,Singaporean,Malaysian,Minority Chinese,Scottish,Arabic,Ukrainian,Chilean,Mongolian,Cuban,Persian,Indonesian,Colombian,Jamaican,Norwegian,Hawaiian,Armenian,Taiwanese,Bangladeshi,Sri Lankan,Cambodian,Albanian,New Zealand,Croatian,Central Asian,Filipino,Tunisian,Cajun & Creole,Romanian,Georgian,Polynesian,Azerbaijani,Caucasian,Afghani,Uzbek,Salvadoran,Yunnan,Native American,Canadian,Xinjiang,Burmese,Fujian,Welsh,Latvian,date_of_Review
0,id_5569,Paris,3.0,5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[12/31/2017, 11/20/2017]"
1,id_1535,Stockholm,1.0,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[07/06/2017, 06/19/2016]"
2,id_352,London,7.0,353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[01/08/2018, 01/06/2018]"
3,id_3456,Berlin,1.0,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[]
4,id_615,Munich,3.0,621

In [58]:
# функция расчета времени от текущего момента до самого раннего отзыва из двух
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return pd.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

# функция расчета времени между отзывами
def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

# с помощью функций заполняем столбец с кол-вом дней то текущего мента 
# c момента самого раннего отзыва из двух
df['day_to_now'] = df.apply(time_to_now, axis = 1).dt.days

# с помощью функций заполняем столбец с кол-вом дней между отзывами
df['day_between_Reviews'] = df.apply(time_between_Reviews, axis = 1).dt.days

<ipython-input-58-9e05b42be0c6>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return pd.datetime.now() - pd.to_datetime(row['date_of_Review']).max()


In [59]:
temp1 = datetime.date.today() - datetime.timedelta(days=df['day_to_now'].min())
temp1

datetime.date(2018, 2, 26)

In [60]:
temp2 = df['day_between_Reviews'].max()
temp2

3207.0

In [61]:
# раз необходимо снова посмотреть на результат МАЕ, то сделам необходимые заполнения и удаления столбцов

df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())

# теперь заполним Price Range
# сначала заменим все значения на числовые 
dic_value_Price = {'$':1,'$$ - $$$':2,'$$$$':3}
df['Price Range']=df['Price Range'].map(lambda x: dic_value_Price.get(x,x))
# потом заменим на самое популярное
df['Price Range'] = df['Price Range'].fillna(df["Price Range"].value_counts().idxmax())

# City переводим в числа с помощью метода get_dummies
df_city_dummies = pd.get_dummies(df['City']).astype('float64')
df = pd.concat([df,df_city_dummies], axis=1)

# заполняем пустые значения в столбцах с днями 
df['day_to_now'] = df['day_to_now'].fillna(df['day_to_now'].max())
df['day_between_Reviews'] = df['day_between_Reviews'].fillna(0)

# удаляем все не нужное
df.drop(['Restaurant_id', 'City', 'Reviews', 'URL_TA', 'ID_TA', 'date_of_Review' ], axis=1, inplace=True)

In [62]:
# проверим
with pd.option_context('display.max_columns', None):
    display(df.describe())

,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,European,French,International,Other,Japanese,Sushi,Asian,Grill,Vegetarian Friendly,Vegan Options,Gluten Free Options,German,Central European,Italian,Pizza,Fast Food,Mediterranean,Spanish,Healthy,Cafe,Thai,Vietnamese,Bar,Pub,Chinese,British,Polish,Fusion,Dutch,Mexican,Venezuelan,South American,Soups,Belgian,Steakhouse,Latin,Barbecue,Argentinean,Irish,Seafood,Swiss,Portuguese,Contemporary,Wine Bar,Greek,Central American,Indian,Middle Eastern,Turkish,Hungarian,Pakistani,Peruvian,Delicatessen,Eastern European,Swedish,Scandinavian,Tibetan,Nepali,Korean,Southwestern,Czech,American,Slovenian,Balti,Street Food,Diner,Brew Pub,Caribbean,Austrian,Moroccan,Halal,Lebanese,Russian,African,Ethiopian,Egyptian,Danish,Brazilian,Ecuadorean,Israeli,Kosher,Gastropub,Australian,Singaporean,Malaysian,Minority Chinese,Scottish,Arabic,Ukrainian,Chilean,Mongolian,Cuban,Persian,Indonesian,Colombian,Jamaican,Norwegian,Hawaiian,Armenian,Taiwanese,Bangladeshi,Sri Lankan,Cambodian,Albanian,New Zealand,Croatian,Central Asian,Filipino,Tunisian,Cajun & Creole,Romanian,Georgian,Polynesian,Azerbaijani,Caucasian,Afghani,Uzbek,Salvadoran,Yunnan,Native American,Canadian,Xinjiang,Burmese,Fujian,Welsh,Latvian,day_to_now,day_between_Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,Budapest,Copenhagen,Dublin,Edinburgh,Geneva,Hamburg,Helsinki,Krakow,Lisbon,Ljubljana,London,Luxembourg,Lyon,Madrid,Milan,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.00000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.0000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,2.622400,3676.028525,3.993037,1.878600,124.825480,0.251500,0.079750,0.03960,0.232075,0.03660,0.028900,0.075275,0.012250,0.279725,0.112150,0.102825,0.016550,0.034825,0.149100,0.071225,0.042625,0.156925,0.069950,0.015500,0.058125,0.018575,0.012825,0.082425,0.061225,0.028625,0.039875,0.009125,0.014425,0.007350,0.011125,0.001000,0.009300,0.012350,0.0

In [63]:
first_model(df)

MAE: 0.21033949999999998
